<a href="https://colab.research.google.com/github/2714830587-xh/CLRS-GG/blob/master/chpter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

一个块可以由许多层组成；一个块可以由许多块组成。

块可以包含代码。

块负责大量的内部处理，包括参数初始化和反向传播。

层和块的顺序连接由Sequential块处理。

In [82]:
import torch
import torch.nn as nn
import torch.nn.functional as F

net=nn.Sequential(nn.Linear(20,10),nn.ReLU(),nn.Linear(10,5))
x=torch.rand(5,20)
net(x)


# simailar
class MLP(nn.Module):
  def __init__(self):
      super(MLP,self).__init__()
      self.lin1=nn.Linear(20,10)
      self.lin2=nn.Linear(10,5)
  def forward(self,x):
    x=F.relu(self.lin1(x))
    x=self.lin2(x)
    return x
net1=MLP()
net(x)



# 仔细地看看Sequential类是如何工作
class MySequential(nn.Module):
  def __init__(self,*args):
      super(MySequential,self).__init__()
      for idx,module in enumerate(args):
        self._modules[str(idx)]=module
  def forward(self,x):
    for block in self._modules.values():
      x=block(x)
    return x
net2=MySequential(nn.Linear(20,10),nn.ReLU(),nn.Linear(10,5))
net2(x)

# 固定参数
class FixedHideenLayer(nn.Module):
  def __init__(self):
      super(FixedHideenLayer,self).__init__()
      self.weight=torch.randn((20,20),requires_grad=False)
      self.lin1=nn.Linear(10,20)
      self.lin2=nn.Linear(20,10)
  def forward(self,x):
    # print(x.shape)
    x=F.relu(torch.mm(x,self.weight)+1)
    # print(x.shape)
    return x
net4=FixedHideenLayer()
x1=torch.randn(20,20)
net4(x1).shape


class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), MySequential())
X=torch.randn(20,20)
print(chimera)
chimera(X).shape


Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=20, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=32, bias=True)
      (3): ReLU()
    )
    (linear): Linear(in_features=32, out_features=16, bias=True)
  )
  (1): Linear(in_features=16, out_features=20, bias=True)
  (2): MySequential()
)


torch.Size([20, 20])

In [29]:
# class MySequential(nn.Module):
#   def init(self, *args):
#     super().init()
#     self.sequential = []
#     for bk in args:
#         self.sequential.append(bk)

#   def forward(self, X):
#     for i in range(len(self.sequential)):
#         X = self.sequential[i](X)
#     return X
# class MySequential2(nn.Module):
#   def init(self, *args):
#     super().init()
#     # self.sequential = []
#     for bk in args:
#       self._modules[bk] = bk

#   def forward(self, X):
#         for bk in self._modules.values():
#             X = bk(X)
#         return X


# X = torch.randn(2, 20)
# # print(net(X))
# # net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
# net2 = MySequential2(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
# print(net(X).shape)
# # print(net2(X).shape)

## 参数管理


当参数绑定时，梯度会发生什么情况？ 答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层 （即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。

In [76]:
import torch
import torch.nn as nn


net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
x=torch.randn(size=(15,4))
net(x).shape

print(net[2].state_dict())
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
net[2].weight.grad == None
#因为还没有调用反向传播



print([(name,param.shape) for name,param in net[0].named_parameters()])
print("test")
print(*[(name,param.shape) for name,param in net.named_parameters()])
#print（*）里面的*号是干嘛的->把序列展开成多个变量


def block1():
  return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())
def block2():
  net=nn.Sequential()
  for i in range(4):
    net.add_module(f'block{i}',block1())
  return net
rgnet=nn.Sequential(block2(),nn.Linear(4,1))

print("rgnet",rgnet)
rgnet(x).shape



# 下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。
rgnet[0][1][0].bias.data



def init_normal(m):
  if type(m)==nn.Linear:
    nn.init.normal_(m.weight,mean=0,std=1)
    nn.init.zeros_(m.bias)
net.apply(init_normal)
# net[0].weight.data[0], net[0].bias.data[0]

# 给定常量初始化
def init_constant(m):
  if type(m)==nn.Linear:
    nn.init.constant_(m.weight,1)
    nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]



# xavier初始化
def xavier(m):
  if type(m)==nn.Linear:
    nn.init.xavier_uniform_(m.weight)
def init_42(m):
  if type(m)==nn.Linear:
    nn.init.constant_(m.weight,42)
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)


def my_init(m):
  if type(m)==nn.Linear:
    print(
        "Init",
        *[(name,param.shape)for name,param in m.named_parameters()][0])
    nn.init.uniform_(m.weight,-10,10)
    m.weight.data*=m.weight.data.abs()>=10
print("my_init")
net.apply(my_init)
print(net[0].weight[:2])

# 直接设置参数
net[0].weight.data[:]+=1
net[0].weight.data[0,0]=42
net[0].weight.data[0]




#参数共享
shared=nn.Linear(8,8)
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),shared,nn.ReLU(),shared,nn.ReLU(),nn.Linear(8,1))
net(x)
print(net[2].weight.data[0]==net[4].weight.data[0])
net[2].weight.data[0,0]=100
print(net[2].weight.data[0]==net[4].weight.data[0])

OrderedDict([('weight', tensor([[-0.0430,  0.2126,  0.2047, -0.0056, -0.0493,  0.1596,  0.0286,  0.0944]])), ('bias', tensor([0.2081]))])
<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2081], requires_grad=True)
tensor([0.2081])
[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
test
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))
rgnet Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_featur

## 自定义层-类似与线性层


In [92]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CenteredLayer(nn.Module):
  def __init__(self):
      super(CenteredLayer,self).__init__()
  def forward(self,x):
    return x-x.mean()
layer=CenteredLayer()
print(layer(torch.FloatTensor([1,2,3,4,5])))
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
# 作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。
Y = net(torch.rand(4, 8))
Y.mean()

# 定义带参数的层
class MyLinear(nn.Module):
  def __init__(self,in_unit,unit):
      super(MyLinear,self).__init__()
      self.weight=nn.Parameter(torch.randn(in_unit,unit))
      self.bias=nn.Parameter(torch.randn(unit,))
  def forward(self,x):
    x=F.relu(torch.mm(x,self.weight)+self.bias)
    return x

linear = MyLinear(5, 3)
linear.weight
linear(torch.rand(2, 5))
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([-2., -1.,  0.,  1.,  2.])


tensor([[0.],
        [0.]], grad_fn=<ReluBackward0>)

## 读写文件-加载和保存模型参数

save和load函数可用于张量对象的文件读写。

我们可以通过参数字典保存和加载网络的全部参数。

保存架构必须在代码中完成，而不是在参数中完成。

In [108]:
import torch
import torch.nn as nn
import torch.nn.functional as F

x=torch.arange(4)
torch.save(x,"x_file")
x2=torch.load("x_file")
x2
# 存储一个张量列表，然后把它们读回内存。
y=torch.zeros(4)
torch.save([x,y],"x_file")
x2,y2=torch.load("x_file")
(x2,y2)


# 我们甚至可以写入或读取从字符串映射到张量的字典 
# 当我们要读取或写入模型中的所有权重时，这很方便。
mydict={"x":x,"y":y}
torch.save(mydict,"my_dict")
mydict2=torch.load("my_dict")
mydict2




class MLP(nn.Module):
  def __init__(self):
      super(MLP,self).__init__()
      self.lin1=nn.Linear(20,256)
      self.lin2=nn.Linear(256,10)
  def forward(self,x):
    return self.lin2(F.relu(self.lin1(x)))
    
net =MLP()
x=torch.randn(20,20)
net(x).shape
y=net(x)
torch.save(net.state_dict(), 'mlp.params')



# 为了恢复模型，我们实例化了原始多层感知机模型的一个备份
clone=MLP()
clone.load_state_dict(torch.load("mlp.params"))
clone.eval()

y_clone=clone(x)
y_clone==y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True,